Model typu filtrowanie kolaboracyjne (collaborative filtering).

Wykorzystujemy w tym celu macierz interakcji, którą następnie dekomponujemy na macierze mniejszej wymiarowości z użyciem 
TruncatedSVD - sklearn. 

In [11]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

Przygotowanie macierzy interakcji

In [12]:
sessionsDataPath = '../notebooks/data/v2/sessions.jsonl'
productsDataPath = '../notebooks/data/v2/products.jsonl'
sessionsDF = pd.read_json(sessionsDataPath, lines=True)
productsDF = pd.read_json(productsDataPath, lines=True)

df = sessionsDF.drop(columns=["session_id", "timestamp", "event_type", "offered_discount", "purchase_id"])
df["count"] = 1

#interaction matrix is build on part of the sessionDF - not perfect way, because test set is all over the place (random shuffel), but i needed something fast for testing
train, test = train_test_split(df, test_size=0.2, stratify=df['product_id'])

interactionMatrixDF = pd.pivot_table(train, index="user_id", columns="product_id", values="count", aggfunc=np.sum, fill_value=0)
display(interactionMatrixDF)

product_id,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319
user_id,,,,,,,,,,,,,,,,,,,,,
102,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
103,0,1,2,14,0,2,2,5,3,7,...,0,1,1,0,0,1,5,1,0,3
104,0,1,5,4,5,6,0,5,3,5,...,1,1,1,0,0,1,7,2,3,1
105,0,1,5,9,1,4,0,2,3,3,...,0,1,1,0,0,0,9,4,0,1
106,2,1,1,7,2,3,2,3,1,4,...,0,0,2,3,1,0,5,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,2,1,6,12,6,2,0,2,5,4,...,0,2,0,2,0,1,9,1,2,3
298,0,0,4,0,2,3,0,4,1,2,...,0,0,0,0,1,2,9,3,0,0
299,0,0,3,0,1,0,0,1,0,1,...,0,0,1,0,2,1,3,1,0,1


Normalizacja wartości w macierzy interakcji.

In [13]:
interactionMatrixDF = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(interactionMatrixDF))
display(interactionMatrixDF)

,0,1,2,3,4,5,6,7,8,9,...,309,310,311,312,313,314,315,316,317,318
0,0.00,0.0,0.071429,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.166667,0.000000,0.000
1,0.00,0.2,0.142857,0.538462,0.000000,0.2,0.5,0.454545,0.3,0.4375,...,0.000000,0.166667,0.2,0.000000,0.000000,0.25,0.192308,0.166667,0.000000,0.375
2,0.00,0.2,0.357143,0.153846,0.416667,0.6,0.0,0.454545,0.3,0.3125,...,0.333333,0.166667,0.2,0.000000,0.000000,0.25,0.269231,0.333333,0.500000,0.125
3,0.00,0.2,0.357143,0.346154,0.083333,0.4,0.0,0.181818,0.3,0.1875,...,0.000000,0.166667,0.2,0.000000,0.000000,0.00,0.346154,0.666667,0.000000,0.125
4,0.25,0.2,0.071429,0.269231,0.166667,0.3,0.5,0.272727,0.1,0.2500,...,0.000000,0.000000,0.4,1.000000,0.142857,0.00,0.192308,0.333333,0.000000,0.125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.25,0.2,0.428571,0.461538,0.500000,0.2,0.0,0.181818,0.5,0.2500,...,0.000000,0.333333,0.0,0.666667,0.000000,0.25,0.346154,0.166667,0.333333,0.375
196,0.00,0.0,0.285714,0.000000,0.166667,0.3,0.0,0.363636,0.1,0.1250,...,0.000000,0.000000,0.0,0.000000,0.142857,0.50,0.346154,0.500000,0.000000,0.000
197,0.00,0.0,0.214286,0.000000,0.083333,0.0,0.0,0.090909,0.0,0.0625,...,0.000000,0.000000,0.2,0.000000,0.285714,0.25,0.115385,0.166667,0.000000,0.125
198,0.25,1.0,0.571429,0.307692,0.250000,0.5,0.0,0.272727,0.3,0.5625,...,0.666667,0.166667,0.0,0.000000,0.571429,1.00,0.307692,0.333333,0.000000,0.375


Utworzenie słownika dla szybszego wyszukiwania.

In [14]:
idxNameDict = pd.Series(productsDF["product_id"].values, index=productsDF.index).to_dict()

Przygotowanie zbirów danych: treningowy i testowy

In [15]:
from sklearn.model_selection import train_test_split

trainUser, testUser = train_test_split(interactionMatrixDF, test_size=0.2)

#transpose because items are columns
# trainItem, testItem = train_test_split(interactionMatrixDF.transpose(), test_size=0.2)
# trainItem = interactionMatrixDF.transpose() #for testing only




Dekompozycja utworzonej macierzy na podmacierze ze względu na użytkowników i produkty.

In [16]:
from sklearn.decomposition import TruncatedSVD

#initial hiperparameters
epsilon = 1e-9
latentFactors = 10

#generate item latent features
itemSVD = TruncatedSVD(n_components=latentFactors)
itemFeatures = itemSVD.fit_transform(interactionMatrixDF.transpose()) + epsilon #transpose because items are columns

#generate user latent features
userSVD = TruncatedSVD(n_components=latentFactors)
userFeatures = userSVD.fit_transform(interactionMatrixDF) + epsilon

pd.DataFrame(userFeatures)
pd.DataFrame(itemFeatures)


,0,1,2,3,4,5,6,7,8,9
0,1.477266,0.106773,0.190632,-0.374573,-0.171057,-0.272680,0.079090,0.362393,-0.069336,0.448465
1,2.717809,0.100473,-0.310654,0.290783,0.259225,-0.638735,0.137034,0.271292,-0.202536,-0.877802
2,3.793310,-0.437570,0.028214,0.381403,0.419213,-0.107690,-0.524893,0.048945,-0.229564,-0.141082
3,3.968510,-0.517629,-0.286436,-0.438662,0.014909,-0.249964,-0.036755,-0.163416,-0.369983,-0.460679
4,4.242310,0.191256,-0.228168,0.234763,-0.448411,0.312692,-0.724924,-0.068717,-0.421900,-0.534361
...,...,...,...,...,...,...,...,...,...,...
314,2.795657,0.720010,-0.094128,0.318552,0.568950,-0.532999,-0.412114,0.411704,-0.112013,-0.903176
315,3.475629,0.491923,-0.043185,0.150967,0.525859,0.301783,-0.429991,0.047248,-0.032754,-0.349988
316,3.459715,0.160406,0.145507,0.548876,0.674354,0.433053,-0.314530,-0.539772,-0.044456,-0.320743
317,2.213423,-0.262426,-0.902582,-0.790464,-0.231635,0.441998,-0.013716,0.410113,-0.147909,0.301837


In [17]:
def top_k(item_id, top_k, corr_mat, map_name):
    topItems = corr_mat[item_id,:].argsort()[-top_k:][::-1]
    topItems = [map_name[e] for e in topItems]
    return topItems

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

itemCorrMat = cosine_similarity(itemFeatures)

#before test_split it is mandatory to create dict mapping indexes of products in productsDF to labels or ids
#because truncatedSVD has rows coresponding to rows in productsDF, but after split there not the same
recommendations = top_k(5, 10, itemCorrMat, idxNameDict)
display(productsDF.loc[productsDF['product_id'].isin(recommendations)])

,product_id,product_name,category_path,price,user_rating
2,1003,Kyocera FS-3640MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,7639.00,1.493143
4,1005,Szalone Króliki Na żywo i w kolorze (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99,2.949198
5,1006,Call of Duty 4 Modern Warfare (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,59.90,3.054270
7,1008,Tom Clancy&#39;s Rainbow Six Vegas (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99,3.715598
8,1009,Kinect Joy Ride (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,69.00,4.801920
10,1011,BioShock Infinite (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,139.99,3.251818
11,1012,Fallout New Vegas (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,69.00,2.386605
12,1013,LA Noire (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,129.99,0.166660
75,1076,Samsung CLX-6260FR ### Gadżety Samsung ### Eks...,Komputery;Drukarki i skanery;Biurowe urządzeni...,2399.00,4.436501
78,1079,Kyocera FS-3040MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,4598.00,0.695480
